In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import tensorflow as tf


## Reading the data

In [3]:
data = pd.read_csv(r"..\..\Data\small_ohe.csv")
data.head()

,age,default,housing,loan,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,...,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
0,0.171429,1,-1,1,0.029412,1.0,0.0,0,0.333333,0.269680,...,0,1,0,0,0,1,0,0,0,0
1,0.300000,1,1,1,0.088235,1.0,0.0,0,0.937500,0.698753,...,0,1,0,0,0,1,0,0,0,0
2,0.100000,1,-1,1,0.000000,1.0,0.0,0,1.000000,0.882307,...,0,0,0,0,0,0,0,0,0,1
3,0.285714,1,0,0,0.058824,1.0,0.0,0,1.000000,0.882307,...,0,0,0,0,0,1,0,0,0,0
4,0.414286,1,-1,1,0.000000,1.0,0.0,0,0.687500,0.389322,...,0,0,1,0,0,0,1,0,0,0


In [4]:
data.columns

Index(['age', 'default', 'housing', 'loan', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y', 'pdays2', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_w

In [5]:
# Saperating features and result vectors
X = data.drop('y', axis=1).values
y = data['y'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [7]:
# Define no sequence of layers
input_tensor = tf.keras.Input(shape=(55,))
layer1 = tf.keras.layers.Dense(64,name = 'layer_1', activation='relu',use_bias=True)(input_tensor)
layer2 = tf.keras.layers.Dense(64,name = 'layer_2', activation='relu',use_bias=True)(layer1)
layer3 = tf.keras.layers.Dense(32,name = 'layer_3', activation='relu',use_bias=True)(layer2)
layer4 = tf.keras.layers.Dense(1, name = 'logit', activation=None,use_bias=True)(layer3)
output_tensor = tf.keras.layers.Activation('sigmoid', name='output')(layer4)
model = tf.keras.Model(inputs=input_tensor, outputs=[layer1,layer2,layer3,layer4,output_tensor]) 
losses = {'logit': 'BinaryCrossentropy'}
model.compile(loss=losses, optimizer='adam', metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, batch_size=128, epochs=200, validation_data=(X_test, y_test))

Epoch 1/200
52/52 [==============================] - 1s 4ms/step - loss: 0.6446 - logit_loss: 0.6446 - layer_1_accuracy: 0.0984 - layer_2_accuracy: 6.1087e-04 - layer_3_accuracy: 0.1365 - logit_accuracy: 0.6423 - output_accuracy: 0.4953 - val_loss: 0.5566 - val_logit_loss: 0.5566 - val_layer_1_accuracy: 0.1113 - val_layer_2_accuracy: 0.0000e+00 - val_layer_3_accuracy: 0.1264 - val_logit_accuracy: 0.7459 - val_output_accuracy: 0.4973
Epoch 2/200
52/52 [==============================] - 0s 1ms/step - loss: 0.5507 - logit_loss: 0.5507 - layer_1_accuracy: 0.0895 - layer_2_accuracy: 0.0000e+00 - layer_3_accuracy: 0.1692 - logit_accuracy: 0.7268 - output_accuracy: 0.4959 - val_loss: 0.5381 - val_logit_loss: 0.5381 - val_layer_1_accuracy: 0.1016 - val_layer_2_accuracy: 0.0000e+00 - val_layer_3_accuracy: 0.1854 - val_logit_accuracy: 0.7569 - val_output_accuracy: 0.5000
Epoch 3/200
52/52 [==============================] - 0s 1ms/step - loss: 0.5054 - logit_loss: 0.5054 - layer_1_accuracy: 0.092

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 55)]              0         
                                                                 
 layer_1 (Dense)             (None, 64)                3584      
                                                                 
 layer_2 (Dense)             (None, 64)                4160      
                                                                 
 layer_3 (Dense)             (None, 32)                2080      
                                                                 
 logit (Dense)               (None, 1)                 33        
                                                                 
 output (Activation)         (None, 1)                 0         
                                                                 
Total params: 9,857
Trainable params: 9,857
Non-trainable par

## Convert NN to ONNX file

In [10]:
import onnxmltools
# convert the model to ONNX format
onnx_net = onnxmltools.convert_keras(model)
onnxmltools.utils.save_model(onnx_net, "my_model.onnx")
content = onnx_net.SerializeToString()

## Setup onnx

In [11]:
import onnxruntime as ort
from sklearn.mixture import GaussianMixture

def setup(onnx_file: str,):
    # Load the ONNX model
    ort_sess = ort.InferenceSession(onnx_file)
    ortvalue = ort.OrtValue.ortvalue_from_numpy(X_train.astype(np.float32), 'cpu')
    logits = ort_sess.run(['logit'], {ort_sess.get_inputs()[0].name: ortvalue})
    outputs = ort_sess.run(['output'], {ort_sess.get_inputs()[0].name: ortvalue})
    x_0 = X_train[y_train == 0].astype(np.float32)
    x_1 = X_train[y_train == 1].astype(np.float32)

    # get predictions for each class with positive true label   
    pred_0 = ort_sess.run(['output'], {ort_sess.get_inputs()[0].name: x_0})[0][:,0]
    x_0 = x_0[pred_0 < 0.5]
    pred_1 = ort_sess.run(['output'], {ort_sess.get_inputs()[0].name: x_1})[0][:,0]
    x_1 = x_1[pred_1 > 0.5]

    logits_0_correct = ort_sess.run(['logit'], {ort_sess.get_inputs()[0].name: x_0})[0]
    logits_1_correct = ort_sess.run(['logit'], {ort_sess.get_inputs()[0].name: x_1})[0]

    # fit GMM to class conditional distributions
    best_bic = 100000000
    best_gmm_0 = GaussianMixture(n_components=1, random_state=0)
    for i in range(1,20):
        gmm_0 = GaussianMixture(n_components=i, random_state=0).fit(logits_0_correct)
        current_bic = gmm_0.bic(logits_0_correct)
        if current_bic < best_bic:
            best_bic = current_bic
            best_gmm_0 = gmm_0

    best_bic = 100000000
    best_gmm_1 = GaussianMixture(n_components=1, random_state=0)
    for i in range(1,20):
        gmm_1 = GaussianMixture(n_components=i, random_state=0).fit(logits_1_correct)
        current_bic = gmm_1.bic(logits_1_correct)
        if current_bic < best_bic:
            best_bic = current_bic
            best_components = i
            best_gmm_1 = gmm_1 
    
    return  best_gmm_0, best_gmm_1, ort_sess

gmm_0, gmm_1, ort_sess = setup('my_model.onnx')

In [12]:
print('gmm_0 number of components is {}'.format(gmm_0.get_params()['n_components']))
print('gmm_1 number of_components is {}'.format(gmm_1.get_params()['n_components']))

gmm_0 number of components is 8
gmm_1 number of_components is 13


In [16]:
q1 = 80
q2 = 60
u1 = 0.5
u2 = 0.2
import math

from sympy import Predicate
def confidence(x, gmm_0, gmm_1, ort_sess):
    logits = ort_sess.run(['logit'], {ort_sess.get_inputs()[0].name: x.astype(np.float32)})[0]
    prediction = ort_sess.run([ort_sess.get_outputs()[-1].name], {ort_sess.get_inputs()[0].name: x.astype(np.float32)})[0]
    prediction = np.squeeze(np.around(prediction, 0))

    u = []
    for gmm in [gmm_0, gmm_1]:
        p = gmm.score_samples(logits)
        s = max(p) - p

        sq1 = np.percentile(s, q1)
        sq2 = np.percentile(s, q2)

        l1 = math.log(1/u1 - 1)
        l2 = math.log(1/u2 - 1)
        c2 = (sq2 * l1 - sq1 * l2) / (l1 - l2) 
        c1 = -l2 / (sq2-c2)

        g = 1/(1 + np.exp(-c1 * (s - c2)))
        u.append(g)
    
    confidence_score = np.zeros([len(x)])
    confidence_score[prediction == 0] = u[0][prediction == 0]
    confidence_score[prediction == 1] = u[1][prediction == 1]

    return prediction, confidence_score

prediction,confidence_score = confidence(X_test, gmm_0, gmm_1, ort_sess)
print('prediction:', prediction)
print('confidence_score:', confidence_score)


prediction: [1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1.
 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1.
 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0.
 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0.
 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1.
 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1.
 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1.